<a href="https://colab.research.google.com/github/mzignis/tensorflow_2.0_tutorial/blob/master/05_transfer_learning_and_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import data and modules

In [1]:
HOME = '/content/drive/My Drive/Colab Notebooks/tensor_flow_2.0'
%cd $HOME

/content/drive/My Drive/Colab Notebooks/tensor_flow_2.0


In [0]:
# !wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip -O ./cats_and_dogs_filtered.zip

In [0]:
import os
import zipfile

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
dataset_path = os.path.join(HOME, 'cats_and_dogs_filtered.zip')

In [0]:
# zip_object = zipfile.ZipFile(file=dataset_path, mode='r')
# zip_object.extractall(path=HOME)
# zip_object.close()

In [0]:
dataset_dir = os.path.join(HOME, 'cats_and_dogs_filtered')
train_dir = os.path.join(dataset_dir, 'train')
validation_dir = os.path.join(dataset_dir, 'validation')

## Building model

### Loading the pre-trained model: MobileNetV2

In [0]:
IMG_SHAPE = (128, 128, 3)

In [0]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

In [9]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

### Freezing model

In [0]:
base_model.trainable = False

### Defining custom head for base model

In [11]:
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [12]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [0]:
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')(global_average_layer)

### Defining transfer learning model

In [14]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

### Compiling transfer learning model

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Creating Data Generators

In [0]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [17]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")
train_generator.labels.shape

Found 2000 images belonging to 2 classes.


(2000,)

In [18]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")
valid_generator.labels.shape

Found 1000 images belonging to 2 classes.


(1000,)

### Training model

In [0]:
model.fit(train_generator, epochs=1, validation_data=valid_generator)

    416/Unknown - 187s 449ms/step - loss: 0.2611 - accuracy: 0.8929